In [2]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Load .env from current working directory (Jupyter notebooks do not define __file__)
env_path = Path().resolve() / ".env"
load_dotenv(dotenv_path=env_path)

# Access variables
OPENAI_API_KEY        = os.getenv("OPENAI_API_KEY")
PINECONE_INDEX_NAME   = os.getenv("PINECONE_INDEX_NAME")
PINECONE_HOST         = os.getenv("PINECONE_HOST")
PINECONE_API_KEY      = os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY        = os.getenv("GEMINI_API_KEY")
K_RETRIEVE            = int(os.getenv("K_RETRIEVE", 5))  # default to 5


In [7]:
import os
import json
from tqdm import tqdm
from typing import List, Tuple

import pinecone
import google.generativeai as genai
from sentence_transformers import SentenceTransformer

# ════════════════ CONFIGS ════════════════
JSON_DIR = "."
EMBEDDING_DIM = 1024
MAX_WORDS = 1000
BATCH_SIZE_EMBED = 32
BATCH_SIZE_UPSERT = 100

# Configura tus claves de API
genai.configure(api_key=GEMINI_API_KEY)

# ════════════════ INIT MODELS ════════════════
model = SentenceTransformer("dariolopez/bge-m3-es-legal-tmp-6")  # 1024-D

# ════════════════ LOAD CORPUS WITH METADATA ════════════════
def load_texts_with_province(json_folder: str) -> Tuple[List[str], List[str]]:
    textos, provincias = [], []
    for filename in sorted(os.listdir(json_folder)):
        if filename.endswith(".json") and filename[:2].isdigit() and 2 <= int(filename[:2]) <= 26:
            filepath = os.path.join(json_folder, filename)
            with open(filepath, "r", encoding="utf-8") as f:
                data = json.load(f)
                for entry in data:
                    if "text" in entry and "province" in entry:
                        textos.append(entry["text"].strip())
                        provincias.append(entry["province"].strip())
    return textos, provincias

ARTICULOS, PROVINCIAS = load_texts_with_province(JSON_DIR)
print(f"📚  Loaded {len(ARTICULOS):,} artículos de {len(set(PROVINCIAS)):,} provincias")

# ════════════════ TRUNCATE LONG TEXTS ════════════════
def truncate_text(text: str, max_words: int = MAX_WORDS) -> str:
    words = text.split()
    return " ".join(words[:max_words])

# ════════════════ EMBEDDING FUNCTION (robusta) ════════════════
def embed_texts(texts: List[str], batch_size: int = BATCH_SIZE_EMBED) -> List[List[float]]:
    all_embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="🧠 Embedding batches"):
        batch_texts = texts[i:i+batch_size]
        formatted = [f"passage: {truncate_text(text)}" for text in batch_texts]
        try:
            embeddings = model.encode(formatted, show_progress_bar=False)
            all_embeddings.extend(embeddings)
        except RuntimeError as e:
            print(f"❌ Error en batch {i}-{i+batch_size}: {e}")
            continue
    return all_embeddings

print("🔧  Generating embeddings …")
EMBEDS = embed_texts(ARTICULOS)
assert len(EMBEDS[0]) == EMBEDDING_DIM, "❌ Embedding dim mismatch!"

# ════════════════ PINECONE SETUP ════════════════
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(name=PINECONE_INDEX_NAME, host=PINECONE_HOST)

def upsert_vectors(texts: List[str],
                   provinces: List[str],
                   vecs: List[List[float]],
                   batch: int = BATCH_SIZE_UPSERT):
    for i in tqdm(range(0, len(texts), batch), desc="⬆️  Upserting"):
        batch_vecs = [
            {
                "id": f"id-{j}",
                "values": vecs[j],
                "metadata": {
                    "text": texts[j],
                    "province": provinces[j]
                }
            }
            for j in range(i, min(i + batch, len(texts)))
        ]
        index.upsert(vectors=batch_vecs)

print("📤  Uploading to Pinecone …")
upsert_vectors(ARTICULOS, PROVINCIAS, EMBEDS)

📚  Loaded 5,689 artículos de 24 provincias
🔧  Generating embeddings …


🧠 Embedding batches:   3%|▎         | 5/178 [01:37<56:30, 19.60s/it]  


KeyboardInterrupt: 

In [ ]:

# ════════════════ RETRIEVE FUNCTION  ════════════════
def retrieve(query: str, k: int = K_RETRIEVE) -> List[str]:
    query_vec = model.encode(f"query: {query}")
    res = index.query(vector=query_vec.tolist(), top_k=k, include_metadata=True)
    return [m.metadata["text"] for m in res.matches]


# ════════════════ GEMINI PRO RAG ════════════════
gemini = genai.GenerativeModel(model_name="gemini-2.0-flash") 

PROMPT_TEMPLATE = """
Eres un/a **abogado/a constitucionalista argentino/a**.  
Tu tarea es **contestar en UNA sola frase** y **exclusivamente** con la
información que aparece dentro de las etiquetas <context></context>.

Reglas de oro (cúmplelas al pie de la letra):

1. Si la respuesta está en el contexto, da la solución **exactamente** como
   figura allí, sin agregar ni quitar nada relevante.
2. Al final de la frase, escribe entre paréntesis el/los número(s) de
   artículo(s) que sustenten la respuesta –por ejemplo: **(art. 14)**.
   - Si el fragmento de contexto trae algo como “Artículo 14 bis”, ponlo igual: **(art. 14 bis)**.
3. Si la información **no** aparece en el contexto, contesta **exactamente**:
   > No tengo información sobre esto.
4. No inventes datos, no cites fuentes externas, no expliques tu razonamiento.
5. Responde en español neutro y evita tecnicismos innecesarios.
6. Si no sabes la respuesta, responde 'no tengo información sobre esto'.

<context>
{context}
</context>

Pregunta: {question}
Respuesta:
""".strip()

def rag_answer(question: str) -> str:
    context = "\n\n".join(retrieve(question))
    prompt  = PROMPT_TEMPLATE.format(context=context, question=question)
    return gemini.generate_content(prompt).text.strip()


In [ ]:
# ════════════════ TEST IT ════════════════
q = "Estoy preparando un negocio de pancho con un puesto en el rio, que me importa de la constitución?"
print("\n🔎 Pregunta:", q)
print("\n🧠 Respuesta (Gemini):\n", rag_answer(q))